# Multi-layer Perceptron Regressor

This is a component that trains a Multi-layer Perceptron Regression model using [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use SDK to load the dataset and save a model.
- how to receive parameters from the platform.

In [ ]:
dataset = "boston" #@param {type:"string"}
target = "medv" #@param {type:"string"}
experiment_id = "c888e8b5-8401-486f-95a3-0c2807d9b0d7" #@param {type:"string"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

## Load metadata
In this context, metadata means information about the dataset.<br>
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
from platiagro import load_metadata
from platiagro.featuretypes import infer_featuretypes

try:
    metadata = load_metadata(name=dataset)
    featuretypes = metadata["featuretypes"]
except KeyError:
    featuretypes = infer_featuretypes(df)

## Encode categorical features

Many machine learning algorithms cannot operate on categorical data directly. They require all input variables and output variables to be numeric.<br>
This means that categorical data must be converted to a numerical form.<br>
The features are converted to ordinal integers. This results in a single column of integers (0 to n_categories - 1) per feature.

In [ ]:
from platiagro.featuretypes import CATEGORICAL
from sklearn.preprocessing import OrdinalEncoder

target_idx = df.columns.tolist().index(target)
# selects the categorical features
categorical_idxs = [idx for idx, ft in enumerate(featuretypes) if ft == CATEGORICAL and idx != target_idx]
feature_encoder = OrdinalEncoder()

if len(categorical_idxs) > 0:
    X[:, categorical_idxs] = feature_encoder.fit_transform(X[:, categorical_idxs])

## Split dataset into train/test splits

Training Dataset: the sample of data used to fit the model.

Test Dataset: the sample of data used to provide an unbiased evaluation of a model fit on the training dataset.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,  train_size=0.7)

## Fit a model using sklearn.neural_network.MLPRegressor

In [ ]:
from sklearn.neural_network import MLPRegressor

estimator = MLPRegressor()
estimator.fit(X_train, y_train)    

## Save model

In [ ]:
from platiagro import save_model

save_model(experiment_id=experiment_id, model={"estimator": estimator, "feature_encoder": feature_encoder})